# CatBoost с массивной генерацией признаков

Пайплайн для:
- Автоматическая генерация сотен признаков
- Полиномиальные признаки, взаимодействия
- Статистические агрегации
- Target encoding, frequency encoding
- CatBoost с автоматическим отбором признаков

In [ ]:
!pip install catboost pandas numpy scikit-learn category_encoders itertools -q

In [ ]:
import pandas as pd
import numpy as np
import catboost as cb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error
from category_encoders import TargetEncoder
from itertools import combinations, combinations_with_replacement
import warnings
warnings.filterwarnings('ignore')

print("✓ Библиотеки загружены!")

## 1. Загрузка данных

In [ ]:
# === ВАШИ ДАННЫЕ ===
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

TARGET_COL = 'target'
ID_COL = 'id'
TASK = 'classification'  # 'classification' или 'regression'

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nTarget distribution:")
print(train_df[TARGET_COL].value_counts())

## 2. Разделение на типы признаков

In [ ]:
# Разделение признаков
numeric_features = train_df.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()

# Удаляем служебные колонки
if ID_COL in numeric_features:
    numeric_features.remove(ID_COL)
if TARGET_COL in numeric_features:
    numeric_features.remove(TARGET_COL)

print(f"Числовых признаков: {len(numeric_features)}")
print(f"Категориальных признаков: {len(categorical_features)}")
print(f"\nЧисловые: {numeric_features[:10]}...")
print(f"Категориальные: {categorical_features[:10]}...")

## 3. Генерация признаков: Числовые взаимодействия

In [ ]:
def generate_numeric_interactions(df, numeric_cols, max_pairs=20):
    """
    Генерация взаимодействий между числовыми признаками
    """
    df = df.copy()
    new_features_count = 0
    
    # Берем топ-N признаков для взаимодействий (чтобы не создать миллионы признаков)
    selected_cols = numeric_cols[:max_pairs]
    
    print(f"Генерация взаимодействий для {len(selected_cols)} признаков...")
    
    # Парные взаимодействия
    for col1, col2 in combinations(selected_cols, 2):
        # Произведение
        df[f'{col1}_mul_{col2}'] = df[col1] * df[col2]
        new_features_count += 1
        
        # Сумма
        df[f'{col1}_add_{col2}'] = df[col1] + df[col2]
        new_features_count += 1
        
        # Разность
        df[f'{col1}_sub_{col2}'] = df[col1] - df[col2]
        new_features_count += 1
        
        # Деление (с защитой от деления на 0)
        df[f'{col1}_div_{col2}'] = df[col1] / (df[col2] + 1e-5)
        df[f'{col2}_div_{col1}'] = df[col2] / (df[col1] + 1e-5)
        new_features_count += 2
        
        # Максимум и минимум
        df[f'{col1}_max_{col2}'] = df[[col1, col2]].max(axis=1)
        df[f'{col1}_min_{col2}'] = df[[col1, col2]].min(axis=1)
        new_features_count += 2
    
    print(f"✓ Создано {new_features_count} новых признаков из взаимодействий")
    return df

train_df = generate_numeric_interactions(train_df, numeric_features, max_pairs=15)
test_df = generate_numeric_interactions(test_df, numeric_features, max_pairs=15)

print(f"Размер после взаимодействий: {train_df.shape}")

## 4. Генерация признаков: Полиномиальные

In [ ]:
def generate_polynomial_features(df, numeric_cols, max_degree=3, max_cols=20):
    """
    Полиномиальные признаки: степени, корни, логарифмы
    """
    df = df.copy()
    new_features_count = 0
    
    selected_cols = numeric_cols[:max_cols]
    print(f"Генерация полиномиальных признаков для {len(selected_cols)} колонок...")
    
    for col in selected_cols:
        # Степени 2, 3
        for degree in range(2, max_degree + 1):
            df[f'{col}_pow{degree}'] = df[col] ** degree
            new_features_count += 1
        
        # Корни
        df[f'{col}_sqrt'] = np.sqrt(np.abs(df[col]))
        df[f'{col}_cbrt'] = np.cbrt(df[col])
        new_features_count += 2
        
        # Логарифмы
        df[f'{col}_log'] = np.log1p(np.abs(df[col]))
        df[f'{col}_log10'] = np.log10(np.abs(df[col]) + 1)
        new_features_count += 2
        
        # Экспонента (с ограничением для стабильности)
        df[f'{col}_exp'] = np.exp(np.clip(df[col], -10, 10))
        new_features_count += 1
        
        # Тригонометрические (если значения в разумном диапазоне)
        if df[col].abs().max() < 100:
            df[f'{col}_sin'] = np.sin(df[col])
            df[f'{col}_cos'] = np.cos(df[col])
            new_features_count += 2
    
    print(f"✓ Создано {new_features_count} полиномиальных признаков")
    return df

train_df = generate_polynomial_features(train_df, numeric_features, max_degree=3, max_cols=15)
test_df = generate_polynomial_features(test_df, numeric_features, max_degree=3, max_cols=15)

print(f"Размер после полиномиальных: {train_df.shape}")

## 5. Генерация признаков: Статистические агрегации

In [ ]:
def generate_statistical_features(df, numeric_cols):
    """
    Статистические признаки по строкам
    """
    df = df.copy()
    
    if len(numeric_cols) == 0:
        return df
    
    print(f"Генерация статистических признаков...")
    
    numeric_data = df[numeric_cols]
    
    # Базовые статистики
    df['row_mean'] = numeric_data.mean(axis=1)
    df['row_std'] = numeric_data.std(axis=1)
    df['row_median'] = numeric_data.median(axis=1)
    df['row_min'] = numeric_data.min(axis=1)
    df['row_max'] = numeric_data.max(axis=1)
    df['row_sum'] = numeric_data.sum(axis=1)
    
    # Производные
    df['row_range'] = df['row_max'] - df['row_min']
    df['row_cv'] = df['row_std'] / (df['row_mean'] + 1e-5)  # Коэффициент вариации
    df['row_skew'] = numeric_data.skew(axis=1)
    df['row_kurtosis'] = numeric_data.kurtosis(axis=1)
    
    # Квантили
    df['row_q25'] = numeric_data.quantile(0.25, axis=1)
    df['row_q75'] = numeric_data.quantile(0.75, axis=1)
    df['row_iqr'] = df['row_q75'] - df['row_q25']
    
    # Количество нулей, положительных, отрицательных
    df['row_zeros'] = (numeric_data == 0).sum(axis=1)
    df['row_positive'] = (numeric_data > 0).sum(axis=1)
    df['row_negative'] = (numeric_data < 0).sum(axis=1)
    
    print(f"✓ Создано {16} статистических признаков")
    return df

# Обновляем список числовых признаков
current_numeric = train_df.select_dtypes(include=[np.number]).columns.tolist()
if ID_COL in current_numeric:
    current_numeric.remove(ID_COL)
if TARGET_COL in current_numeric:
    current_numeric.remove(TARGET_COL)

train_df = generate_statistical_features(train_df, numeric_features)
test_df = generate_statistical_features(test_df, numeric_features)

print(f"Размер после статистических: {train_df.shape}")

## 6. Генерация признаков: Категориальные

In [ ]:
def generate_categorical_features(train, test, cat_cols, target_col):
    """
    Frequency encoding, target encoding, комбинации категорий
    """
    train = train.copy()
    test = test.copy()
    
    if len(cat_cols) == 0:
        return train, test
    
    print(f"Генерация категориальных признаков для {len(cat_cols)} колонок...")
    new_features_count = 0
    
    # 1. Frequency Encoding
    for col in cat_cols:
        freq_map = train[col].value_counts().to_dict()
        train[f'{col}_freq'] = train[col].map(freq_map)
        test[f'{col}_freq'] = test[col].map(freq_map).fillna(0)
        new_features_count += 1
    
    # 2. Количество уникальных значений (для строк)
    for col in cat_cols:
        train[f'{col}_nunique'] = train[col].apply(lambda x: len(str(x)))
        test[f'{col}_nunique'] = test[col].apply(lambda x: len(str(x)))
        new_features_count += 1
    
    # 3. Комбинации категориальных признаков
    if len(cat_cols) >= 2:
        for col1, col2 in combinations(cat_cols[:5], 2):  # Ограничиваем 5-ю для скорости
            train[f'{col1}_{col2}_combo'] = train[col1].astype(str) + '_' + train[col2].astype(str)
            test[f'{col1}_{col2}_combo'] = test[col1].astype(str) + '_' + test[col2].astype(str)
            new_features_count += 1
    
    # 4. Target Encoding (для высокой кардинальности)
    high_card_cols = [col for col in cat_cols if train[col].nunique() > 10]
    if len(high_card_cols) > 0:
        target_encoder = TargetEncoder(cols=high_card_cols)
        train[high_card_cols] = target_encoder.fit_transform(train[high_card_cols], train[target_col])
        test[high_card_cols] = target_encoder.transform(test[high_card_cols])
        print(f"   Target encoding применен к {len(high_card_cols)} колонкам")
    
    # 5. Label Encoding для низкой кардинальности
    low_card_cols = [col for col in cat_cols if train[col].nunique() <= 10]
    for col in low_card_cols:
        le = LabelEncoder()
        train[col] = le.fit_transform(train[col].astype(str))
        test[col] = le.transform(test[col].astype(str))
    
    print(f"✓ Создано {new_features_count}+ категориальных признаков")
    return train, test

train_df, test_df = generate_categorical_features(train_df, test_df, categorical_features, TARGET_COL)

print(f"Размер после категориальных: {train_df.shape}")

## 7. Обработка пропусков и inf

In [ ]:
# Замена inf на NaN
train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Заполнение пропусков медианой
numeric_cols_all = train_df.select_dtypes(include=[np.number]).columns.tolist()
if ID_COL in numeric_cols_all:
    numeric_cols_all.remove(ID_COL)
if TARGET_COL in numeric_cols_all:
    numeric_cols_all.remove(TARGET_COL)

for col in numeric_cols_all:
    if train_df[col].isnull().sum() > 0:
        median_val = train_df[col].median()
        train_df[col].fillna(median_val, inplace=True)
        test_df[col].fillna(median_val, inplace=True)

print(f"✓ Пропуски обработаны")
print(f"Финальный размер train: {train_df.shape}")
print(f"Финальный размер test: {test_df.shape}")
print(f"Всего создано признаков: {train_df.shape[1] - len(train_df.columns) + len(numeric_features) + len(categorical_features)}")

## 8. Подготовка данных для CatBoost

In [ ]:
# Выделяем признаки и таргет
feature_cols = [col for col in train_df.columns if col not in [TARGET_COL, ID_COL]]

X = train_df[feature_cols]
y = train_df[TARGET_COL]
X_test = test_df[feature_cols]

# Train/Val split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42,
    stratify=y if TASK == 'classification' else None
)

print(f"Количество признаков: {len(feature_cols)}")
print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

## 9. Обучение CatBoost

In [ ]:
# Параметры CatBoost
catboost_params = {
    'iterations': 2000,
    'learning_rate': 0.03,
    'depth': 8,
    'l2_leaf_reg': 3,
    'random_seed': 42,
    'verbose': 200,
    'early_stopping_rounds': 100,
    'task_type': 'GPU' if cb.cuda.is_cuda_available() else 'CPU',
}

if TASK == 'classification':
    catboost_params['loss_function'] = 'Logloss'
    catboost_params['eval_metric'] = 'AUC'
    model = cb.CatBoostClassifier(**catboost_params)
else:
    catboost_params['loss_function'] = 'RMSE'
    catboost_params['eval_metric'] = 'RMSE'
    model = cb.CatBoostRegressor(**catboost_params)

print(f"Обучение CatBoost с {len(feature_cols)} признаками...")

# Обучение
model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    use_best_model=True,
    verbose=200
)

print("\n✓ Обучение завершено!")

## 10. Feature Importance